# Search Algorithm Comparison

Here can compare the performance and costs for each of the aforementioned search algorithms. The benchmarking function returns the algorithm's name, the runtime of the algorithm (in seconds), the route's cost (in metres), as well as the algorithm's search space.

In [2]:
# Setup the Graph
import osmnx
import pandas
from smart_mobility_utilities.common import Node
from smart_mobility_utilities.search import *

reference = (43.661667, -79.395)
G = osmnx.graph_from_point(reference, dist=500, clean_periphery=True, simplify=True)
origin = Node(graph=G, osmid=1907446268)
destination = Node(graph=G, osmid=1633421938)

stats = []
# Benchmark Blind Search Algorithms
stats.append(benchmark(BFS,use_G=False, G=G,origin=origin, destination=destination)) # BFS
stats.append(benchmark(DFS,use_G=False, G=G,origin=origin, destination=destination)) # DFS
stats.append(benchmark(dijkstra, G=G,origin=origin, destination=destination)) # Dijkstra

# Benchmark Informed Search Algorithms
stats.append(benchmark(hill_climbing, G=G, origin=origin, destination=destination))
stats.append(benchmark(beam, G=G, origin=origin, destination=destination))
stats.append(benchmark(astar,G=G,origin=origin, destination=destination))
stats.append(benchmark(bidirectional_astar,G=G,origin=origin, destination=destination))
stats.append(('Bidirectional-Dijkstra with contraction','***','***'))
df = pandas.DataFrame(stats, columns=['Algorithm','Time','Cost (m)'])
df.style.hide_index() # Hide the index for display purposes
df.round(3)

,Algorithm,Time,Cost (m)
0,BFS,0.009988,1385.116
1,DFS,0.02271,6092.076
2,dijkstra,0.132459,1029.481
3,hill_climbing,15.410266,1232.897
4,beam,97.033216,1073.653
5,astar,0.006665,1137.69
6,bidirectional_astar,0.012139,1137.69
7,Bidirectional-Dijkstra with contraction,***,***


:::{note}
The runtime for `Bidirectional-Dijkstra with contraction` represents the final search time only. The amount of time spent preprocessing the data used for this search (approximately 1,100 nodes) exceeded 39 hours on a single CPU.
:::

## Comparison of Time and Space Complexities

| Algorithm                               | Worst-case Time          | Worst-case Space | Completeness | Optimal |
|-----------------------------------------|--------------------------|------------------|--------------|---------|
| BFS                                     | $O(b^d)$                 | $O(b^d)$         | Yes          | No      |
| DFS                                     | $O(b^d)$                 | $O(bd)$          | Yes          | No      |
| Dijkstra                                | $O(\|E\|+\|V\|log\|V\|)$ | $O(b^d)$         | Yes          | Yes     |
| Hill Climbing                           | $O(\infty)$&#10013;      | $O(b)$           | No           | No      |
| Beam Search                             | $O(\infty)$&#10013;      | $O(kb)$          | No           | No      |
| A* Search                               | $O(b^d)$                 | $O(b^d)$         | Yes          | No*     |
| Bidirectional A*                        | $O(b^d)$                 | $O(b^d/2)$       | Yes          | No*     |
| Bidirectional Dijkstra with contraction | **                       | **               | Yes          | Yes     |

\* An A* Heuristic search can be "optimally efficient", depending on the heuristic used. <br>
&#10013; As both Hill Climbing and Beam Search can get stuck at local maxima, they are incomplete. <br>
\*\* Contraction-based bidirectional searches perform at the same time complexity as a normal bidirectional Dijkstra, at the expense of drastically increased preprocessing time.<br>
<br>

At first glance, it may seem thata Bidirectional Dijkstra is the "best" search algorithm, as it is both complete and optimal, as well as more space efficient than a normal Dijkstra search. However, when dealing with large datasets, space complexity is often a bigger hurdle than actual performance or guaranteeing an optimal solution.

When comparing search algorithms, it is important to be aware of the constraints for any given problem, and select an algorithm based on those constraints. For example, certain implementations of hill climbing may result in rapid exit conditions. If the goal is to maximize number of problems solved (and local maxima are an acceptable result), Hill Climbing algorithms result in quick solutions that have some degree of "optimality".

On the other hand, preprocessing-heavy algorithms like contraction hierarchies offer incredibly low space costs (even more so when implemented with a bidirectional search), as well as rapid searches for guaranteed optimal solutions (if using Dijkstra). For high-volume usage implementations where preprocessing is not a concern (i.e. Uber), contraction hierarchies are a good choice. In fact, the `osrm` package that is used in this book is primarily based on an implementation of contraction hierarchies.